# 导入模块

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time, datetime 

import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [3]:
pd.set_option('display.max_columns', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

# 定义数据库连接

In [4]:
def query(sql,
          host="rm-wz930e5269fur1ht1mo.mysql.rds.aliyuncs.com",
          user="wxz",
          password="5JRcY9SaiepVlIq7iuPo",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 获取订单数据

In [5]:
sql1 = ''' -- 订单&风控信息  近10日数据   
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,cc.name as channel_name         -- 来源渠道
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name
,topay.total_freeze_fund_amount 
,om.buy_service_product,tso.status as service_status 
,om.order_method
from  db_digua_business.t_order  om 
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 
where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静'
,'陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')
-- and COALESCE(pa.name, '未知') not in ("1000单秘密计划","1000单秘密计划-无优惠","1000单曙光计划","线下门店3个月试行") 
and  om.create_time >= DATE_ADD(CURRENT_DATE, INTERVAL -1 month) 
-- and  om.create_time <= DATE_ADD(CURRENT_DATE, INTERVAL -12 day)        -- 整月数据
;
'''

df_order = query(sql1)
df_order.shape

(144347, 29)

# 合并数据

In [6]:
# df = df_order.merge(df_order_risk,left_on='订单id',right_on='order_id',how='left')
df = df_order.copy()
df.order_id.notnull().sum()

np.int64(144347)

# 处理日期

In [7]:

df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["下单日期"]
df["月份"]=df["下单日期"].dt.month
df['create_hour']=df["create_time"].dt.hour
df.shape

(144347, 32)

In [8]:
df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')
df['拒绝理由'].value_counts()

拒绝理由
                                                                                     67352
冰鉴火眸分<500命中强拒                                                                        26164
命中强拒加入临时黑名单                                                                           9964
年龄超过49岁或低于18岁                                                                         3963
命中自有模型强拒                                                                              2360
                                                                                     ...  
命中特殊策略2强拒, 命中策略240703_4强拒, 命中自有模型强拒, 蚁盾分>=80                                             1
命中策略240703_14强拒, 命中策略240703_4强拒, 评分等级低于D,命中拒绝                                            1
命中云商分低于496强拒, 命中自有模型强拒, 命中被执行人或曝光台, 命中青云分<500                                            1
为法院被执行人且未全部结案, 命中云商分低于496强拒, 命中策略240703_4强拒, 命中自有模型强拒, 命中融安分低于680强拒, 评分等级低于D,命中拒绝        1
命中特殊策略2强拒, 命中策略240703_4强拒, 命中自有模型强拒, 命中被执行人或曝光台                                      

# 处理备注信息

In [9]:
df = df[df['sku_attributes'].notnull()]

In [10]:

df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].astype(str).str.split("$").str[0].str.strip()
df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].astype(str).str.split("$").str[0].str.strip()
df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].astype(str).str.split("$").str[0].str.strip()
df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")

def getcolor(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="颜色":
            return color_list[j]["value"]
df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1)
df["颜色"].value_counts()  

# %%
def getneicun(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="内存":
            return color_list[j]["value"]
df.loc[:,"内存"]=df.apply(lambda x:getneicun(x["sku_attributes"]),axis=1)
df["内存"].value_counts() 

df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
df["进件"].value_counts()

# %%
df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")

In [11]:
df['免审'] = np.where(df.decision_result.str.contains(pat='免人审',regex=False),1,0)
df['免审'].value_counts()

免审
0    138158
1      6189
Name: count, dtype: int64

# 归属渠道

In [12]:
def qudao_type(a,b,c):
    a = str(a)
    b = str(b)
    if "租物" in b:
        return "芝麻租物"
    elif "芝麻" in b:
        return "芝麻租物"
    elif "抖音" in b:
        return "抖音渠道"
    elif "搜索" in a:
        return "搜索渠道"
    elif "租物" in a:
        return "芝麻租物"
    elif "芝麻" in a:
        return "芝麻租物" 
    elif c == 1:
        return "芝麻租物"  
    elif "叮咚直播" in a:
        return "叮咚直播"
    elif "租瓜直播2号" in a:
        return "租瓜直播2号"
    elif "租瓜直播" in a:
        return "租瓜直播"
    elif "直播" in a:
        return "支付宝直播"
    elif "繁星" in a:
        return "繁星"
    elif "生活号" in a:
        return "生活号"
    elif "群" in a:
        return "支付宝社群"
    elif "付费灯火" in a:
        return "付费灯火"
    else :
        return a
    

# %%
df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["activity_name"],x["order_method"]),axis=1)
df["归属渠道"].value_counts()

归属渠道
芝麻租物                              78588
搜索渠道                              44675
单人聊天会话中的小程序消息卡片（分享）                9127
九州信息                               2006
我的小程序入口                            1850
其他渠道场景渠道。                          1297
生活号                                1275
小程序商家消息（服务提醒）                      1232
未知渠道                                828
抖音渠道                                674
支付宝社群                               643
支付宝直播                               446
支付宝 push 消息（同1014）                  333
订阅消息                                252
付费邦道-首页                             225
付费侠客行-苹果旗舰                          193
付费流量(通过商家数字推广平台，灯火等投放的广告)           152
付费灯火                                103
侠客行vivo                              84
扫描二维码                                78
邦道月活BD02                             75
广州易合顺                                48
系统桌面图标                               29
线下易拉宝                                23
支付宝客户端首页                           

# 订单去重

In [13]:
dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["status2"].map(dict_status_code)


df.sort_values(by=["下单日期","状态编码"],inplace=True)


#删除重复单号
df.drop_duplicates(subset=["order_id"],inplace=True)
#删除身份证空值行
df.dropna(subset=["id_card_num"],axis=0,inplace=True)
#去刷单订单
df.drop(df[df['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['activity_name']=="线下门店3个月试行"].index,inplace=True)
#删除身份证空值行
df.dropna(subset=["status2"],axis=0,inplace=True)


#删除重复订单
df.drop_duplicates(subset=["order_id"],inplace=True)
df.drop_duplicates(subset=["true_name","user_mobile","id_card_num","下单日期"],keep="last",inplace=True)


df.drop(df[df['true_name'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文"])].index,inplace=True)

# 定义状态

In [14]:
# 判断 前置拦截   机审强拒   出库前风控强拒
df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)


In [15]:
def reject_type(a,b,c,d,e,f,g,h,i,j):
    if h==1 and b=="未进件":
        return "前置拦截"
    elif i==1 and b=="进件":
        return "机审强拒"    
    #抖音渠道
    # elif h==1 and b=="进件":
    #     return "机审强拒"  
    elif  b=="进件" and c is not np.nan and '已退款' in e:
        return "人审拒绝"
    elif  b=="进件" and d is not np.nan:
        return "客户取消"
    elif  b=="进件" and f is not np.nan:
        return "无法联系"
    elif  b=="进件" and j==1 :
        return "出库前风控强拒"
    elif  b=="进件" and "待审核" in e:
        return "待审核"
    elif  b=="进件" and "待发货" in e:
        return "出库"
    elif  b=="进件" and "待收货" in e:
        return "出库"
    elif b=="进件" and "租赁中" in e:
        return "出库"
    elif  b=="进件" and "已完成" in e:
        return "出库"
    elif a is np.nan and b=="未进件":
        return "未进件"
    elif  b=="未进件":
        return "进件前取消"
    else :
        return e

df.loc[:,"审核状态"]=df.apply(lambda x: reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)#,x["无法联系原因"]
df["审核状态"].value_counts()  

审核状态
进件前取消    50143
前置拦截     24544
机审强拒     11237
人审拒绝      8516
客户取消      3890
Name: count, dtype: int64

In [16]:
df_contain = df.copy()

# 剔除商家数据

In [17]:
# #剔除商家数据只保留自营租机业务数据
# df.drop(df[df['merchant_name']=="深圳优优大数据科技有限公司"].index,inplace=True)
# df.drop(df[df['merchant_name']=="优优2店"].index,inplace=True)
# df.drop(df[df['merchant_name']=="小豚租（代收）"].index,inplace=True)
# df.drop(df[df['merchant_name']=="苏州蚁诺宝"].index,inplace=True)
# df.drop(df[df['merchant_name']=="租着用电脑数码"].index,inplace=True)
# df.drop(df[df['merchant_name']=="北京海鸟窝科技有限公司"].index,inplace=True)

# df.drop(df[df['merchant_name']=="汇客好租"].index,inplace=True)
# df.drop(df[df['merchant_name']=="澄心优租"].index,inplace=True)
# df.drop(df[df['merchant_name']=="CPS渠道合作"].index,inplace=True)
# df.drop(df[df['sku_attributes'].str.contains(pat='探路者',regex=False)==True].index,inplace=True)  
# # 趣智数码  单
# df.drop(df[df['merchant_name']=="趣智数码"].index,inplace=True)
# df.drop(df[df['merchant_name']=="格木木二奢名品"].index,inplace=True)
# df.drop(df[df['merchant_name']=="广州康基贸易有限公司"].index,inplace=True)
# df.drop(df[df['merchant_name']=="汇客好租"].index,inplace=True)
# df.drop(df[df['merchant_name']=="线下小店"].index,inplace=True)
# df.drop(df[df['merchant_name']=="小蚂蚁租机"].index,inplace=True)

# # 
# df.drop(df[df['merchant_name'].str.contains(pat='探路者',regex=False)==True].index,inplace=True) 
df = clean.drop_merchant(df)

# 去重订单明细

In [18]:
from datetime import datetime  
today = datetime.today() 
t_date = today.date().strftime('%Y-%m-%d')
t_date

'2025-01-08'

# 导入服务订单

In [19]:
df["下单时段"]=df["create_time"].astype(str).str[:14]

# %%
df_j = df[df["进件"]=="进件"]
df_j.shape




(22393, 53)

In [20]:
df["待审核"] = np.where(df["审核状态"]=='待审核',1,0)
df["前置拦截"] = np.where(df["审核状态"]=='前置拦截',1,0)
df["机审强拒"] = np.where(df["审核状态"]=='机审强拒',1,0)
df["人审拒绝"] = np.where(df["审核状态"]=='人审拒绝',1,0)
df["客户取消"] = np.where(df["审核状态"]=='客户取消',1,0)
df["出库前风控强拒"] = np.where(df["审核状态"]=='出库前风控强拒',1,0)
df["无法联系"] = np.where(df["审核状态"]=='无法联系',1,0)
df["是否进件"] = np.where(df["进件"]=='进件',1,0)
df["是否出库"] = np.where(df["status"].isin([2,3,4,5,6,8,15]),1,0)

df["进件前取消"] = np.where(df["审核状态"]=='进件前取消',1,0)

df = df[df['审核状态']!='已退款']
df['是否出库'] = np.where((df['人审拒绝']==0)&(df['客户取消']==0)&(df['无法联系']==0)&(df['待审核']==0)&(df['是否出库']==1), 1, 0)

In [21]:
# dfdf_ck = df[df["是否出库"]==1]

# 导入台账数据

In [22]:
#读取导入出库台账数据
f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2023&2024年台账.xlsx"
# wb = pd.ExcelFile(f_path_ck)  
# df_ck = pd.read_excel(wb, sheet_name="2023")
df_ck = pd.read_excel(f_path_ck,sheet_name="2023")   #,skiprows=0 header 
df_ck.shape

(27111, 19)

In [23]:
df_ck['溢价费订单'] = "A"+df_ck[df_ck['备注'].str.contains(pat='溢价费',regex=False)==True]['备注'].str.extract('(\d+.\d+)')
df_yijia = df_ck[df_ck['溢价费订单'].str.len()>=16][['溢价费订单',"已付金额"]]
df_yijia=df_yijia.rename(columns={'溢价费订单':"单号","已付金额":"溢价费"})
df_yijia_ck = pd.merge(df_ck,df_yijia,left_on="订单号", right_on="单号",how="left")
dfck = pd.merge(df_yijia_ck,df,left_on="订单号", right_on="order_number")

dfck.drop_duplicates(subset=["订单号"],inplace=True)
dfck.shape#.to_excel(f"C:/Users/Administrator/Desktop/报表模板/出库台账最终整理/11月出库补充1220.xlsx")

(2990, 85)

In [24]:
dfck[dfck["status2"]=="已退款"]

dfck.drop(dfck[dfck["status2"]=="已退款"].index,inplace=True)

dfck[dfck["status2"]=="已退款"]

# 删除 露营设备 出库 
dfck.drop(dfck[dfck["类目"]=="露营设备"].index,inplace=True)


# 八派信息

In [25]:
#搜索
df0 = df[df["来源渠道"]=="八派信息"]
# df0['下单日期']=='2024-09-03'
df_group = df0.groupby(df0['下单日期']).agg({'order_id':'size','是否进件':'sum','前置拦截':'sum','机审强拒':'sum','人审拒绝':'sum','客户取消':'sum','无法联系':'sum','出库前风控强拒':'sum','待审核':'sum','是否出库':'sum'})
df_group.rename(columns={'order_id':'去重订单数'},inplace=True )

df_group["拦截率"] = df_group["前置拦截"]/df_group["去重订单数"]
df_group["拦截率"] = df_group["拦截率"].apply(lambda x:format(x,".2%"))

df_group['预授权通过率'] = df_group["是否进件"]/df_group["去重订单数"]
df_group["预授权通过率"] = df_group["预授权通过率"].apply(lambda x:format(x,".2%"))

df_group["强拒比例"] = df_group["机审强拒"]/df_group["是否进件"]
df_group["强拒比例"] = df_group["强拒比例"].apply(lambda x:format(x,".2%"))

df_group["机审通过件"] = df_group["是否进件"]-df_group["机审强拒"]

df_group["风控通过件"] = df_group["是否进件"]-df_group["机审强拒"]-df_group["人审拒绝"]
df_group["风控通过率"] = df_group["风控通过件"]/df_group["是否进件"]
df_group["风控通过率"] = df_group["风控通过率"].apply(lambda x:format(x,".2%"))

df_group["取消率"] = df_group["客户取消"]/df_group["是否进件"]
df_group["取消率"] = df_group["取消率"].apply(lambda x:format(x,".2%"))

#df_group["无法联系占比"]=df_group["无法联系"]/df_group["是否进件"]
#df_group["无法联系占比"]=df_group["无法联系占比"].apply(lambda x:format(x,".2%"))

df_group["人审拒绝率"]=df_group["人审拒绝"]/df_group["是否进件"]
df_group["人审拒绝率"]=df_group["人审拒绝率"].apply(lambda x:format(x,".2%"))

df_group["进件出库率"]=df_group["是否出库"]/df_group["是否进件"]
df_group["进件出库率"]=df_group["进件出库率"].apply(lambda x:format(x,".2%"))

df_group["订单出库率"]=df_group["是否出库"]/df_group["去重订单数"]
df_group["订单出库率"]=df_group["订单出库率"].apply(lambda x:format(x,".2%"))

df_group[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核", "是否出库"]]                


# df_ss.to_excel("F:/myfile/业务相关数据/搜索渠道数据转化0617.xlsx")    


,去重订单数,前置拦截,拦截率,是否进件,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,是否出库
下单日期,,,,,,,,,,,,,,,,


# 九州信息

In [26]:
#搜索
df0 = df[df["来源渠道"]=="九州信息"]
df_group = df0.groupby(df0['下单日期']).agg({'order_id':'size','是否进件':'sum','前置拦截':'sum','机审强拒':'sum','人审拒绝':'sum','客户取消':'sum','无法联系':'sum','出库前风控强拒':'sum','待审核':'sum','是否出库':'sum'})
df_group.rename(columns={'order_id':'去重订单数'},inplace=True )

df_group["拦截率"] = df_group["前置拦截"]/df_group["去重订单数"]
df_group["拦截率"] = df_group["拦截率"].apply(lambda x:format(x,".2%"))

df_group['预授权通过率'] = df_group["是否进件"]/df_group["去重订单数"]
df_group["预授权通过率"] = df_group["预授权通过率"].apply(lambda x:format(x,".2%"))

df_group["强拒比例"] = df_group["机审强拒"]/df_group["是否进件"]
df_group["强拒比例"] = df_group["强拒比例"].apply(lambda x:format(x,".2%"))

df_group["机审通过件"] = df_group["是否进件"]-df_group["机审强拒"]

df_group["风控通过件"] = df_group["是否进件"]-df_group["机审强拒"]-df_group["人审拒绝"]
df_group["风控通过率"] = df_group["风控通过件"]/df_group["是否进件"]
df_group["风控通过率"] = df_group["风控通过率"].apply(lambda x:format(x,".2%"))

df_group["取消率"] = df_group["客户取消"]/df_group["是否进件"]
df_group["取消率"] = df_group["取消率"].apply(lambda x:format(x,".2%"))

#df_group["无法联系占比"]=df_group["无法联系"]/df_group["是否进件"]
#df_group["无法联系占比"]=df_group["无法联系占比"].apply(lambda x:format(x,".2%"))

df_group["人审拒绝率"]=df_group["人审拒绝"]/df_group["是否进件"]
df_group["人审拒绝率"]=df_group["人审拒绝率"].apply(lambda x:format(x,".2%"))

df_group["进件出库率"]=df_group["是否出库"]/df_group["是否进件"]
df_group["进件出库率"]=df_group["进件出库率"].apply(lambda x:format(x,".2%"))

df_group["订单出库率"]=df_group["是否出库"]/df_group["去重订单数"]
df_group["订单出库率"]=df_group["订单出库率"].apply(lambda x:format(x,".2%"))

df_group = df_group[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核", "是否出库"]]                
df_group.rename(columns={'是否进件': '进件', '是否出库': '出库'}, inplace=True)
Today1 = datetime.now().strftime('%Y%m%d')
# with pd.ExcelWriter(f"F:/myfile/归属渠道商家进件信息/九州信息_{Today1}.xlsx", engine='openpyxl') as writer:
#     df_group.to_excel(writer, sheet_name='九州信息') 



# 每日各渠道去重订单统计 导出

In [27]:
today = datetime.today() 
t_date = today.date().strftime('%Y-%m-%d')
t_date

'2025-01-08'

# 按周

## 整天去重订单统计

In [28]:
# 计算环比
def calculate_growth_rate(current, previous):
    if previous == 0:
        return float('inf') if current > 0 else float('-inf')
    return (current - previous) / previous

In [29]:
qd_type_list = ['下单日期', '付费灯火', '八派信息', '单人聊天会话中的小程序消息卡片（分享）', '我的小程序入口', '搜索渠道', '支付宝社群', '生活号', '芝麻租物']
weekday = 4
gsqd_qcdd = pd.crosstab(df["下单日期"],df['归属渠道'],margins=True)
qd_type_name = df['归属渠道'].drop_duplicates()[df['归属渠道'].drop_duplicates().isin(qd_type_list)].to_list()
qd_type_name
gsqd_qcdd = gsqd_qcdd[qd_type_name][:-1].reset_index()
gsqd_qcdd['下单日期'] = pd.to_datetime(gsqd_qcdd['下单日期'])
    
gsqd_qcdd = gsqd_qcdd[gsqd_qcdd['下单日期'].dt.dayofweek+1==weekday]

# 创建新的DataFrame对象存储新数据
gsqd_qcdds = pd.DataFrame({
    '下单日期': [f'本周{weekday}与上周{weekday}对比增长数量', f'本周{weekday}与上周{weekday}对比增长率']
})
gsqd_qcdd

归属渠道,下单日期,芝麻租物,搜索渠道,单人聊天会话中的小程序消息卡片（分享）,我的小程序入口,支付宝社群,生活号,付费灯火
4,2024-12-12,1553,1107,235,48,7,45,13
11,2024-12-19,1061,1342,297,50,20,48,0
18,2024-12-26,1356,1274,234,34,15,23,0
25,2025-01-02,971,1268,268,41,16,15,0


In [30]:
# 设置需要周几的数据
weekday = 4
qd_type_list = ['下单日期', '付费灯火', '八派信息', '单人聊天会话中的小程序消息卡片（分享）', '我的小程序入口', '搜索渠道', '支付宝社群', '生活号', '芝麻租物']
qd_list = ['归属渠道', '来源渠道']

def qcdd(qd_name):
    #每日各渠道去重订单统计结果表格导出
    gsqd_qcdd = pd.crosstab(df["下单日期"],df[qd_name],margins=True)
    # 获取各个渠道
    qd_type_name = df[qd_name].drop_duplicates()[df[qd_name].drop_duplicates().isin(qd_type_list)].to_list()
    gsqd_qcdd = gsqd_qcdd[qd_type_name][:-1].reset_index()
    gsqd_qcdd['下单日期'] = pd.to_datetime(gsqd_qcdd['下单日期'])
    
    gsqd_qcdd = gsqd_qcdd[gsqd_qcdd['下单日期'].dt.dayofweek+1==weekday]

    # 创建新的DataFrame对象存储新数据
    gsqd_qcdds = pd.DataFrame({
        '下单日期': [f'本周{weekday}与上周{weekday}对比增长数量', f'本周{weekday}与上周{weekday}对比增长率']
    })
    # 循环写入数据
    for type_list in qd_type_name:
        try:
            gsqd_qcdds[type_list] = [gsqd_qcdd[type_list].diff().iloc[-1],  (gsqd_qcdd[type_list].pct_change() * 100).apply(lambda x: f'{x:.2f}%').iloc[-1]]
        except:
            gsqd_qcdds[type_list] = [0, 0]
    # gsqd_qcdds
    gsqd_qcddss = pd.concat([gsqd_qcdd, gsqd_qcdds])
    

    return gsqd_qcddss

# 使用函数
gsqd_qcddss = qcdd(qd_list[0])
gsqd_qcddss



,下单日期,芝麻租物,搜索渠道,单人聊天会话中的小程序消息卡片（分享）,我的小程序入口,支付宝社群,生活号,付费灯火
4,2024-12-12 00:00:00,1553,1107,235,48,7,45,13
11,2024-12-19 00:00:00,1061,1342,297,50,20,48,0
18,2024-12-26 00:00:00,1356,1274,234,34,15,23,0
25,2025-01-02 00:00:00,971,1268,268,41,16,15,0
0,本周4与上周4对比增长数量,-385.0,-6.0,34.0,7.0,1.0,-8.0,0.0
1,本周4与上周4对比增长率,-28.39%,-0.47%,14.53%,20.59%,6.67%,-34.78%,nan%


## 按小时去重订单统计

In [31]:
qd_type_lists = ['create_time', '付费灯火', '八派信息', '单人聊天会话中的小程序消息卡片（分享）', '我的小程序入口', '搜索渠道', '支付宝社群', '生活号', '芝麻租物']
# 截止日期
hour = 23
def qcdd_h(qd_name):
    #每日各渠道去重订单统计结果表格导出
    gsqd_qcdd = pd.crosstab(df["create_time"],df[qd_name],margins=True)
    # 获取各个渠道
    qd_type_name = df[qd_name].drop_duplicates()[df[qd_name].drop_duplicates().isin(qd_type_lists)].to_list()
    gsqd_qcdd = gsqd_qcdd[qd_type_name][:-1].reset_index()
    gsqd_qcdd['create_time'] = pd.to_datetime(gsqd_qcdd['create_time'])
    gsqd_qcdd['日期'] = gsqd_qcdd['create_time'].dt.date
    gsqd_qcdd['小时'] = gsqd_qcdd['create_time'].dt.hour
    gsqd_qcdd['日期'] = pd.to_datetime(gsqd_qcdd['日期'])

    gsqd_qcdd = gsqd_qcdd[gsqd_qcdd['日期'].dt.dayofweek+1==weekday]
    gsqd_qcdd_group = gsqd_qcdd.groupby(['日期', '小时'])[qd_type_name].sum()
    gsqd_qcdd_group = gsqd_qcdd_group.query(f'小时<={hour}')

    gsqd_qcdd_groups = gsqd_qcdd_group.groupby(['日期'])[qd_type_name].cumsum()
    gsqd_qcdd_groups = gsqd_qcdd_groups.groupby(['日期'])[qd_type_name].last()

    gsqd_qcdd_groups = gsqd_qcdd_groups.reset_index()
    gsqd_qcdd_groups.insert(1, '小时', hour)
    gsqd_qcdd_groups

    gsqd_qcdds = pd.DataFrame({
        '日期': [f'本周{weekday}与上周{weekday}对比增长数量', f'本周{weekday}与上周{weekday}对比增长率'],
        '小时': [' ', '']
    })
    for type_lists in qd_type_name:
        try:
            gsqd_qcdds[type_lists] = [gsqd_qcdd_groups[type_lists].diff().iloc[-1],  (gsqd_qcdd_groups[type_lists].pct_change() * 100).apply(lambda x: f'{x:.2f}%').iloc[-1]]
        except:
            gsqd_qcdds[type_lists] = [0, 0]
    gsqd_qcdds = pd.concat([gsqd_qcdd_groups, gsqd_qcdds])

    return gsqd_qcdds


In [32]:
gsqd_qcdd_group = qcdd_h(qd_list[0])
gsqd_qcdd_group
Today1 = datetime.now().strftime('%Y%m%d')
with pd.ExcelWriter(f'F:/myfile/归属渠道商家进件信息/数据增长渠道对比分析_{Today1}.xlsx', engine='openpyxl') as writer:
    gsqd_qcddss.to_excel(writer, sheet_name=f'每周{weekday}_截止至{hour}_归属渠道去重订单统计', index=False)

## 整天进件统计

In [33]:

def jj(qd_name):
    #每日各渠道去重订单统计结果表格导出
    gsqd_jj = pd.crosstab(df["下单日期"],df_j[qd_name],margins=True)
    qd_type_name = df[qd_name].drop_duplicates()[df[qd_name].drop_duplicates().isin(qd_type_list)].to_list()
    gsqd_jj = gsqd_jj[qd_type_name][:-1].reset_index()
    gsqd_jj['下单日期'] = pd.to_datetime(gsqd_jj['下单日期'])

    gsqd_jj = gsqd_jj[gsqd_jj['下单日期'].dt.dayofweek+1==weekday]

    gsqd_jjs = pd.DataFrame({
        '下单日期': [f'本周{weekday}与上周{weekday}对比增长数量', f'本周{weekday}与上周{weekday}对比增长率']
    })
    for type_list in qd_type_name:
        try:
            gsqd_jjs[type_list] = [gsqd_jj[type_list].diff().iloc[-1],  (gsqd_jj[type_list].pct_change() * 100).apply(lambda x: f'{x:.2f}%').iloc[-1]]
        except:
            gsqd_jjs[type_list] = [0, 0]

    gsqd_jjss = pd.concat([gsqd_jj, gsqd_jjs])

    return gsqd_jjss

gsqd_jj_group = jj(qd_list[0])
gsqd_jj_group

,下单日期,芝麻租物,搜索渠道,单人聊天会话中的小程序消息卡片（分享）,我的小程序入口,支付宝社群,生活号,付费灯火
4,2024-12-12 00:00:00,260,328,54,3,0,6,5
11,2024-12-19 00:00:00,227,387,63,10,2,9,0
18,2024-12-26 00:00:00,288,389,69,8,3,3,0
25,2025-01-02 00:00:00,226,349,64,7,2,5,0
0,本周4与上周4对比增长数量,-62.0,-40.0,-5.0,-1.0,-1.0,2.0,0.0
1,本周4与上周4对比增长率,-21.53%,-10.28%,-7.25%,-12.50%,-33.33%,66.67%,nan%


## 按小时进件统计

In [34]:
def jj_h(qd_name):
    #每日各渠道去重订单统计结果表格导出
    gsqd_jj = pd.crosstab(df["create_time"],df_j[qd_name],margins=True)
    qd_type_name = df[qd_name].drop_duplicates()[df[qd_name].drop_duplicates().isin(qd_type_lists)].to_list()
    gsqd_jj = gsqd_jj[qd_type_name][:-1].reset_index()
    gsqd_jj['create_time'] = pd.to_datetime(gsqd_jj['create_time'])
    gsqd_jj['日期'] = gsqd_jj['create_time'].dt.date
    gsqd_jj['小时'] = gsqd_jj['create_time'].dt.hour
    gsqd_jj['日期'] = pd.to_datetime(gsqd_jj['日期'])

    gsqd_jj = gsqd_jj[gsqd_jj['日期'].dt.dayofweek+1==weekday]

    gsqd_jj_group = gsqd_jj.groupby(['日期', '小时'])[qd_type_name].sum()

    gsqd_jj_group = gsqd_jj_group.query(f'小时<={hour}')
    gsqd_jj_groups = gsqd_jj_group.groupby(['日期'])[qd_type_name].cumsum()
    gsqd_jj_groups = gsqd_jj_groups.groupby(['日期'])[qd_type_name].last()

    gsqd_jj_groups = gsqd_jj_groups.reset_index()
    gsqd_jj_groups.insert(1, '小时', hour)

    gsqd_jjs = pd.DataFrame({
        '日期': [f'本周{weekday}与上周{weekday}对比增长数量', f'本周{weekday}与上周{weekday}对比增长率'],
        '小时': [' ', '']
    })

    for type_lists in qd_type_name:
        try:
            gsqd_jjs[type_lists] = [gsqd_jj_groups[type_lists].diff().iloc[-1],  (gsqd_jj_groups[type_lists].pct_change() * 100).apply(lambda x: f'{x:.2f}%').iloc[-1]]
        except:
            gsqd_jjs[type_lists] = [0, 0]
    gsqd_jjs = pd.concat([gsqd_jj_groups, gsqd_jjs])

    return gsqd_jjs

In [35]:
gsqd_jjs = jj_h(qd_list[0])



# gsqd_jjs
with pd.ExcelWriter(f'F:/myfile/归属渠道商家进件信息/数据增长渠道对比分析_{Today1}.xlsx', engine='openpyxl', mode='a') as writer:
    gsqd_jjs.to_excel(writer, sheet_name=f'每周{weekday}_截止至{hour}_归属渠道进件统计', index=False)
gsqd_jjs

,日期,小时,芝麻租物,搜索渠道,单人聊天会话中的小程序消息卡片（分享）,我的小程序入口,支付宝社群,生活号,付费灯火
0,2024-12-12 00:00:00,23,260,328,54,3,0,6,5
1,2024-12-19 00:00:00,23,227,387,63,10,2,9,0
2,2024-12-26 00:00:00,23,288,389,69,8,3,3,0
3,2025-01-02 00:00:00,23,226,349,64,7,2,5,0
0,本周4与上周4对比增长数量,,-62.0,-40.0,-5.0,-1.0,-1.0,2.0,0.0
1,本周4与上周4对比增长率,,-21.53%,-10.28%,-7.25%,-12.50%,-33.33%,66.67%,nan%


In [36]:

with pd.ExcelWriter(f'F:/myfile/归属渠道商家进件信息/数据增长渠道对比分析_{Today1}.xlsx', engine='openpyxl', mode='a') as writer:
    gsqd_qcdd_group.to_excel(writer, sheet_name=f'总表_归属渠道去重订单统计')


In [37]:
with pd.ExcelWriter(f'F:/myfile/归属渠道商家进件信息/数据增长渠道对比分析_{Today1}.xlsx', engine='openpyxl', mode='a') as writer:
    gsqd_jj_group.to_excel(writer, sheet_name=f'总表_归属渠道进件统计')

In [38]:
with pd.ExcelWriter(f'F:/myfile/归属渠道商家进件信息/数据增长渠道对比分析_{Today1}.xlsx', engine='openpyxl', mode='a') as writer:
    gsqd_qcddss.to_excel(writer,sheet_name=f'每周{weekday}_归属渠道去重订单统计', index=False)

In [39]:
with pd.ExcelWriter(f'F:/myfile/归属渠道商家进件信息/数据增长渠道对比分析_{Today1}.xlsx', engine='openpyxl', mode='a') as writer:
    gsqd_jjs.to_excel(writer, sheet_name=f'每周{weekday}_归属渠道进件统计', index=False)

In [40]:
gsqd_jj_group

,下单日期,芝麻租物,搜索渠道,单人聊天会话中的小程序消息卡片（分享）,我的小程序入口,支付宝社群,生活号,付费灯火
4,2024-12-12 00:00:00,260,328,54,3,0,6,5
11,2024-12-19 00:00:00,227,387,63,10,2,9,0
18,2024-12-26 00:00:00,288,389,69,8,3,3,0
25,2025-01-02 00:00:00,226,349,64,7,2,5,0
0,本周4与上周4对比增长数量,-62.0,-40.0,-5.0,-1.0,-1.0,2.0,0.0
1,本周4与上周4对比增长率,-21.53%,-10.28%,-7.25%,-12.50%,-33.33%,66.67%,nan%


In [41]:
#每日各渠道去重订单统计结果表格导出
qd_list = ['create_time', '付费灯火', '九州信息', '单人聊天会话中的小程序消息卡片（分享）', '我的小程序入口', '搜索渠道', '支付宝社群', '生活号', '芝麻租物']
gsqd_jj = pd.crosstab(df['create_time'], df['归属渠道'],margins=True)

qd_names = df['来源渠道'].drop_duplicates()[df['来源渠道'].drop_duplicates().isin(qd_list)].to_list()

gsqd_jj = gsqd_jj.reset_index()
gsqd_jj = gsqd_jj[qd_list][:-1]
gsqd_jj['create_time'] = pd.to_datetime(gsqd_jj['create_time'])
gsqd_jj['日期'] = gsqd_jj['create_time'].dt.date
gsqd_jj['小时'] = gsqd_jj['create_time'].dt.hour
gsqd_jj_groups = pd.DataFrame()

gsqd_jj_group = gsqd_jj.groupby(['日期', '小时'])[qd_names].sum()

gsqd_jj_group

KeyError: "['八派信息'] not in index"

# 按天

In [41]:
# 计算环比
def calculate_growth_rate(current, previous):
    if previous == 0:
        return float('inf') if current > 0 else float('-inf')
    return (current - previous) / previous

In [1]:
#每日各渠道去重订单统计结果表格导出
qd_list = ['create_time', '付费灯火', '九州信息', '单人聊天会话中的小程序消息卡片（分享）', '我的小程序入口', '搜索渠道', '支付宝社群', '生活号', '芝麻租物', '小程序商家消息（服务提醒）', '支付宝 push 消息（同1014）']
# qd_list = ['create_time', '付费灯火', '八派信息', '单人聊天会话中的小程序消息卡片（分享）', '我的小程序入口', '搜索渠道', '生活号', '芝麻租物'] “小程序商家消息”“支付宝push”
gsqd_jj = pd.crosstab(df['create_time'], df['归属渠道'],margins=True)

qd_names = df['归属渠道'].drop_duplicates()[df['归属渠道'].drop_duplicates().isin(qd_list)].to_list()
qd_namess = df['归属渠道'].drop_duplicates()[df['归属渠道'].drop_duplicates().isin(qd_list)].to_list()
qd_namess.append('create_time')

gsqd_jj = gsqd_jj.reset_index()
gsqd_jj = gsqd_jj[qd_namess][:-1]

NameError: name 'pd' is not defined

In [42]:
def df_all(qd_date, qd_name, hour, minute):
    #每日各渠道去重订单统计结果表格导出
    qd_list = ['create_time', '付费灯火', '九州信息', '单人聊天会话中的小程序消息卡片（分享）', '我的小程序入口', '搜索渠道', '支付宝社群', '生活号', '芝麻租物', '小程序商家消息（服务提醒）', '支付宝 push 消息（同1014）']
    # qd_list = ['create_time', '付费灯火', '八派信息', '单人聊天会话中的小程序消息卡片（分享）', '我的小程序入口', '搜索渠道', '生活号', '芝麻租物'] “小程序商家消息”“支付宝push”
    gsqd_jj = pd.crosstab(qd_date, qd_name,margins=True)

    qd_names = qd_name.drop_duplicates()[qd_name.drop_duplicates().isin(qd_list)].to_list()
    qd_namess = qd_name.drop_duplicates()[qd_name.drop_duplicates().isin(qd_list)].to_list()
    qd_namess.append('create_time')

    gsqd_jj = gsqd_jj.reset_index()
    gsqd_jj = gsqd_jj[qd_namess][:-1]
    gsqd_jj['create_time'] = pd.to_datetime(gsqd_jj['create_time'])
    gsqd_jj['日期'] = gsqd_jj['create_time'].dt.date
    gsqd_jj['小时'] = gsqd_jj['create_time'].dt.hour
    gsqd_jj_groups = pd.DataFrame()

    gsqd_jj_group = gsqd_jj.groupby(['日期', '小时'])[qd_names].sum()
    # 定义截止时间到小时分钟
    gsqd_jj['下单时间'] = pd.to_datetime(gsqd_jj['create_time'])
    gsqd_jj['下单日期'] = gsqd_jj['下单时间'].dt.date
    # hour = 18
    # minute = '00'
    # minute = 30
    # qd_names = qd_name.drop_duplicates()[qd_name.drop_duplicates().isin(qd_list)].to_list()

    # 定义时间阈值
    time_threshold = pd.to_datetime(f'{hour}:{minute}').time()
    # 按 '下单日期' 分组
    gsqd_jj = gsqd_jj.groupby('下单日期')
    # 初始化一个空的 DataFrame 来存储结果
    result_df = pd.DataFrame()
    # 遍历每个组并筛选小于 15:30 的数据
    for date, group in gsqd_jj:
        filtered_group = group[group['下单时间'].dt.time <= time_threshold]
        result_df = pd.concat([result_df, filtered_group], ignore_index=True)
    gsqd_jj = result_df
    
    # 截止日期
    days=14
    # hour = 9
    # 当前小时
    # hour = datetime.now().strftime('%H')
    gsqd_jj_group = gsqd_jj_group.reset_index()
    gsqd_jj_group['日期'] = pd.to_datetime(gsqd_jj_group['日期'])
    three_ago = pd.to_datetime(datetime.now().strftime('%Y-%m-%d'))-timedelta(days=days)
    three_after = pd.to_datetime(datetime.now().strftime('%Y-%m-%d'))-timedelta(days=0)
    gsqd_jj_group = gsqd_jj_group[gsqd_jj_group['日期']>=three_ago]
    # gsqd_jj_group = gsqd_jj_group[(gsqd_jj_group['日期']>=three_ago)&(gsqd_jj_group['日期']<=three_after)]
    gsqd_jj_group = gsqd_jj_group.query(f'小时<{hour}')
    # gsqd_jj_group = gsqd_jj_group.iloc[-(int(hour)+1)*3:]
    
    gsqd_jj_group.set_index('日期', inplace=True)
    gsqd_jj_groups = gsqd_jj_group.groupby(['日期'])[qd_names].cumsum()
    gsqd_jj_groups = gsqd_jj_groups.groupby(['日期'])[qd_names].last()
    gsqd_jj_groups.insert(0, '小时', hour)
    gsqd_jj_groups = gsqd_jj_groups.reset_index()
    gsqd_jj_groups

    first = gsqd_jj_groups['日期'].iloc[0].strftime('%m%d')
    current = gsqd_jj_groups['日期'].iloc[-1].strftime('%m%d')
    previous = gsqd_jj_groups['日期'].iloc[-2].strftime('%m%d')


    # 创建新的df对象存储数据
    gsqd_jjs = pd.DataFrame({
        '日期': [f'{current}与{previous}对比增长数量', f'{current}与{previous}对比增长率'],
        '小时': [' ', ' ']
    })
    for qd in qd_names:
        try:
            gsqd_jjs[qd] = [gsqd_jj_groups[qd].diff().iloc[-1],  (gsqd_jj_groups[qd].pct_change() * 100).apply(lambda x: f'{x:.0f}%').iloc[-1]]
        except:
            gsqd_jjs[qd] = [0, 0]
    gsqd_jjs = pd.concat([gsqd_jj_groups, gsqd_jjs])

    return gsqd_jj_group, gsqd_jjs, first, current


In [43]:
qd_list = ['归属渠道', '来源渠道']
indx = 0
gsqd_jj_group1, gsqd_jjs1, first, current = df_all(df['create_time'], df[qd_list[indx]], 18, '00')
gsqd_jj_group3, gsqd_jjs3, first, current = df_all(dfck['create_time'], dfck[qd_list[indx]], 18, '00')
gsqd_jj_group2, gsqd_jjs2, first, current = df_all(df_j['create_time'], df_j[qd_list[indx]], 18, '00')
gsqd_jjs3


,日期,小时,芝麻租物,搜索渠道,单人聊天会话中的小程序消息卡片（分享）,生活号,小程序商家消息（服务提醒）,我的小程序入口,九州信息,付费灯火,支付宝社群
0,2024-12-04 00:00:00,18,30,56,5,0,0,1,3,0,0
1,2024-12-05 00:00:00,18,55,29,5,0,0,2,1,0,0
2,2024-12-06 00:00:00,18,56,32,1,0,0,3,1,0,1
3,2024-12-07 00:00:00,18,48,31,1,0,0,1,0,0,0
4,2024-12-08 00:00:00,18,38,27,1,1,0,2,0,0,0
5,2024-12-09 00:00:00,18,57,35,2,0,0,1,2,0,0
6,2024-12-10 00:00:00,18,70,54,3,0,0,1,1,0,0
7,2024-12-11 00:00:00,18,47,38,5,0,0,0,0,0,1
8,2024-12-12 00:00:00,18,45,48,5,0,0,1,1,2,0
9,2024-12-13 00:00:00,18,45,42,4,0,0,1,2,0,0


# 分期乐

In [44]:
# ["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒",
# "待审核", "是否出库"]]                

# df[df['来源渠道'].str.contains('分期乐')]['来源渠道'].drop_duplicates()
df_fql = df[df['来源渠道'].str.contains('分期乐')]
df_fql_group = df_fql.groupby('来源渠道').agg({'order_id':'size','是否进件':'sum','前置拦截':'sum','机审强拒':'sum','人审拒绝':'sum','客户取消':'sum','无法联系':'sum','出库前风控强拒':'sum','待审核':'sum','是否出库':'sum'})
df_fql_group.rename(columns={'order_id':'去重订单数'},inplace=True )

df_fql_group["拦截率"] = df_fql_group["前置拦截"]/df_fql_group["去重订单数"]
df_fql_group["拦截率"] = df_fql_group["拦截率"].apply(lambda x:format(x,".2%"))

df_fql_group['预授权通过率'] = df_fql_group["是否进件"]/df_fql_group["去重订单数"]
df_fql_group["预授权通过率"] = df_fql_group["预授权通过率"].apply(lambda x:format(x,".2%"))

df_fql_group["强拒比例"] = df_fql_group["机审强拒"]/df_fql_group["是否进件"]
df_fql_group["强拒比例"] = df_fql_group["强拒比例"].apply(lambda x:format(x,".2%"))

df_fql_group["机审通过件"] = df_fql_group["是否进件"]-df_fql_group["机审强拒"]

df_fql_group["风控通过件"] = df_fql_group["是否进件"]-df_fql_group["机审强拒"]-df_fql_group["人审拒绝"]
df_fql_group["风控通过率"] = df_fql_group["风控通过件"]/df_fql_group["是否进件"]
df_fql_group["风控通过率"] = df_fql_group["风控通过率"].apply(lambda x:format(x,".2%"))

df_fql_group["取消率"] = df_fql_group["客户取消"]/df_fql_group["是否进件"]
df_fql_group["取消率"] = df_fql_group["取消率"].apply(lambda x:format(x,".2%"))

#df_fql_group["无法联系占比"]=df_fql_group["无法联系"]/df_group["是否进件"]
#df_fql_group["无法联系占比"]=df_fql_group["无法联系占比"].apply(lambda x:format(x,".2%"))

df_fql_group["人审拒绝率"]=df_fql_group["人审拒绝"]/df_fql_group["是否进件"]
df_fql_group["人审拒绝率"]=df_fql_group["人审拒绝率"].apply(lambda x:format(x,".2%"))

df_fql_group["进件出库率"]=df_fql_group["是否出库"]/df_fql_group["是否进件"]
df_fql_group["进件出库率"]=df_fql_group["进件出库率"].apply(lambda x:format(x,".2%"))

df_fql_group["订单出库率"]=df_fql_group["是否出库"]/df_fql_group["去重订单数"]
df_fql_group["订单出库率"]=df_fql_group["订单出库率"].apply(lambda x:format(x,".2%"))

df_fql_group = df_fql_group[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核", "是否出库"]]                

# with pd.ExcelWriter('F:/myfile/归属渠道商家进件信息/分期乐.xlsx', engine='openpyxl') as writer:
#     df_fql_group.to_excel(writer, sheet_name='来源渠道')

In [45]:
# ["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒",
# "待审核", "是否出库"]]                

# df[df['来源渠道'].str.contains('分期乐')]['来源渠道'].drop_duplicates()
df_fql = df[df['归属渠道'].str.contains('分期乐')]
df_fql_group = df_fql.groupby('归属渠道').agg({'order_id':'size','是否进件':'sum','前置拦截':'sum','机审强拒':'sum','人审拒绝':'sum','客户取消':'sum','无法联系':'sum','出库前风控强拒':'sum','待审核':'sum','是否出库':'sum'})
df_fql_group.rename(columns={'order_id':'去重订单数'},inplace=True )

df_fql_group["拦截率"] = df_fql_group["前置拦截"]/df_fql_group["去重订单数"]
df_fql_group["拦截率"] = df_fql_group["拦截率"].apply(lambda x:format(x,".2%"))

df_fql_group['预授权通过率'] = df_fql_group["是否进件"]/df_fql_group["去重订单数"]
df_fql_group["预授权通过率"] = df_fql_group["预授权通过率"].apply(lambda x:format(x,".2%"))

df_fql_group["强拒比例"] = df_fql_group["机审强拒"]/df_fql_group["是否进件"]
df_fql_group["强拒比例"] = df_fql_group["强拒比例"].apply(lambda x:format(x,".2%"))

df_fql_group["机审通过件"] = df_fql_group["是否进件"]-df_fql_group["机审强拒"]

df_fql_group["风控通过件"] = df_fql_group["是否进件"]-df_fql_group["机审强拒"]-df_fql_group["人审拒绝"]
df_fql_group["风控通过率"] = df_fql_group["风控通过件"]/df_fql_group["是否进件"]
df_fql_group["风控通过率"] = df_fql_group["风控通过率"].apply(lambda x:format(x,".2%"))

df_fql_group["取消率"] = df_fql_group["客户取消"]/df_fql_group["是否进件"]
df_fql_group["取消率"] = df_fql_group["取消率"].apply(lambda x:format(x,".2%"))

#df_fql_group["无法联系占比"]=df_fql_group["无法联系"]/df_group["是否进件"]
#df_fql_group["无法联系占比"]=df_fql_group["无法联系占比"].apply(lambda x:format(x,".2%"))

df_fql_group["人审拒绝率"]=df_fql_group["人审拒绝"]/df_fql_group["是否进件"]
df_fql_group["人审拒绝率"]=df_fql_group["人审拒绝率"].apply(lambda x:format(x,".2%"))

df_fql_group["进件出库率"]=df_fql_group["是否出库"]/df_fql_group["是否进件"]
df_fql_group["进件出库率"]=df_fql_group["进件出库率"].apply(lambda x:format(x,".2%"))

df_fql_group["订单出库率"]=df_fql_group["是否出库"]/df_fql_group["去重订单数"]
df_fql_group["订单出库率"]=df_fql_group["订单出库率"].apply(lambda x:format(x,".2%"))

df_fql_group = df_fql_group[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核", "是否出库"]]                

# with pd.ExcelWriter('F:/myfile/归属渠道商家进件信息/分期乐.xlsx', engine='openpyxl', mode='a') as writer:
#     df_fql_group.to_excel(writer, sheet_name='归属渠道')


# 定时任务

In [46]:
from apscheduler.schedulers.blocking import BlockingScheduler
from apscheduler.schedulers.background import BackgroundScheduler

hour = 18
minute = 00
# H = datetime.now().strftime('%H')
# M = datetime.now().strftime('%M')
def my_job1():
    print(f'执行定时任务：现在是每日的{hour}点{minute}分')
    # print(f'执行定时任务：现在是每日的{H}点{int(M)+1}分')
    path = r'\\digua\迪瓜租机\19.小程序发货率/'
    Today1 = str(datetime.now().strftime('%Y%m%d%H'))
    with pd.ExcelWriter(path + f'各渠道增长对比分析_{first}-{current}.xlsx', engine='openpyxl') as writer:
        gsqd_jjs1.to_excel(writer, sheet_name=f'{qd_list[indx]}去重订单统计(汇总)', index=False)
    
    with pd.ExcelWriter(path + f'各渠道增长对比分析_{first}-{current}.xlsx', engine='openpyxl', mode='a') as writer:
        gsqd_jjs2.to_excel(writer, sheet_name=f'{qd_list[indx]}进件统计(汇总)', index=False)
        gsqd_jjs3.to_excel(writer, sheet_name=f'{qd_list[indx]}出库统计(汇总)', index=False)
        gsqd_jj_group1.to_excel(writer, sheet_name=f'{qd_list[indx]}去重订单统计')
        gsqd_jj_group2.to_excel(writer, sheet_name=f'{qd_list[indx]}进件统计')
        gsqd_jj_group3.to_excel(writer, sheet_name=f'{qd_list[indx]}出库统计')
scheduler1 = BackgroundScheduler()
# 添加一个cron任务，每天的10:30执行
scheduler1.add_job(my_job1, 'cron', hour=hour, minute=minute)

# scheduler1.add_job(my_job1, 'cron', hour=H, minute=int(M)+1)

# 启动scheduler1
scheduler1.start()




执行定时任务：现在是每日的18点0分
